<a href="https://colab.research.google.com/github/duskvirkus/colab-notebooks/blob/main/gangealing/gangealing_colab_vis_correspondence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unofficial Colab Notebook for GAN-Supervised Dense Visual Alignment

Paper Authors: William Peebles and Jun-Yan Zhu and Richard Zhang and Antonio Torralba and Alexei Efros and Eli Shechtman

Notebook by [@duskvirkus](https://github.com/duskvirkus)

See https://github.com/wpeebles/gangealing for more info.

Things to know:

- Currently only setup to run vis_correspondence.py script but might add more in the furture.
- Takes ~3 hours to get things up and running before you can experiment on one dataset. (most of the time is spent downloading a dataset)
- Requires Google Drive account with 100+ of free space.

Notebook last updated: Dec 15 2021

## Setup

Recommend running this section and the preprocessing section without a gpu to avoid annoying colab (which may or may not effect what gpus you get on colab). Then once you reach the following sections switch and rerun this setup section.

**Important** after running the first cell `Runtime > Restart Runtime`

In [2]:
!pip install "jupyterlab>=3" "ipywidgets>=7.6"

     |████████████████████████████████| 8.6 MB 11.4 MB/s 
     |████████████████████████████████| 58 kB 8.1 MB/s 
     |████████████████████████████████| 428 kB 64.9 MB/s 
     |████████████████████████████████| 396 kB 71.2 MB/s 
     |████████████████████████████████| 53 kB 2.8 MB/s 
     |████████████████████████████████| 78 kB 9.5 MB/s 
     |████████████████████████████████| 129 kB 77.0 MB/s 
     |████████████████████████████████| 69 kB 9.4 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 2.6.0
    Uninstalling jsonschema-2.6.0:
      Successfully uninstalled jsonschema-2.6.0
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5
ERROR: pip's dependency resolver doe

In [1]:
# Drive is required because the dataset won't fit on a colab instance and this notebook required restarting frequently
# Notebook reuires about 100 GB of free drive space (mostly for a dataset will go up if you use multiple)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
drive_path = '/content/drive/MyDrive/'
repo_container_dir = 'colab-gangealing'
repo_name = 'gangealing'
git_repo = 'https://github.com/wpeebles/gangealing.git'
branch_name = 'main'

working_dir = os.path.join(drive_path, repo_container_dir, repo_name)

if os.path.isdir(working_dir):
  %cd {working_dir}
else:
  container_path = os.path.join(drive_path, repo_container_dir)
  os.makedirs(container_path)
  %cd {container_path}
  !git clone --branch {branch_name} {git_repo}
  %cd {repo_name}
  !mkdir data

/content/drive/MyDrive/colab-gangealing/gangealing


In [3]:
!pip install moviepy plotly==5.4.0 tensorboard>=1.15 tqdm lmdb ray ninja termcolor

In [4]:
os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + ':' + os.getcwd() # same as export PYTHONPATH="${PYTHONPATH}:${PWD}" which doesn't work in colab

In [5]:
# Dataset Variables
dataset_name = 'tv-monitor' # options are [bicycle, cat, dog, tv-monitor] there are more in the repo but they will need some more work
dataset_path_zip = os.path.join(working_dir, 'data', dataset_name + '.zip')
dataset_path_zip_md5 = os.path.join(working_dir, 'data', dataset_name + '.zip.md5')
dataset_path = os.path.join(working_dir, 'data', dataset_name)
dataset_out = os.path.join(working_dir, 'data', 'lsun_' + dataset_name)
ckpt_name = ''.join(dataset_name.split('-'))
mask_path = os.path.join(working_dir, 'assets', 'masks', ckpt_name + '_mask.png')

## Preprocess Data

In [ ]:
# this cell will likely take ~2 hours
%cd data
!wget http://dl.yf.io/lsun/objects/{dataset_name}.zip
!wget http://dl.yf.io/lsun/objects/{dataset_name}.zip.md5
%cd {working_dir}

In [ ]:
# takes a couple minutes
!cat {dataset_path_zip_md5}
!md5sum {dataset_path_zip}

In [ ]:
# takes a couple minutes
%cd data
!unzip {dataset_name + '.zip'}
%cd {working_dir}

In [7]:
!python prepare_data.py --input_is_lmdb --path {dataset_path} --out {dataset_out} --size 512 --max_images 10000

Make dataset of image sizes: 512
Loading LMDB keys (this might take a bit)...
LMDB keys loaded! Found 10000 keys.
100% 10000/10000 [12:17<00:00, 13.56it/s]
Final dataset size: 10000


## vis_correspondence

At this point switch notebook to GPU runtime by going to `Runtime > Change runtime type > Hardware accelerator > GPU`. This will restart the runtime so rerun the setup section.


In [6]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-f99156f8-2622-8e17-a8c9-f854b218e87d)


In [7]:
!python applications/vis_correspondence.py \
  --ckpt {ckpt_name} \
  --real_data_path {dataset_out} \
  --vis_in_stages \
  --real_size 512 \
  --output_resolution 512 \
  --resolution 512 \
  --label_path {mask_path} \
  --dset_indices 2 # change this number for different images in dataset

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3784704/45929032 bytes (8.2%)7864320/45929032 bytes (17.1%)11837440/45929032 bytes (25.8%)15982592/45929032 bytes (34.8%)19914752/45929032 bytes (43.4%)24141824/45929032 bytes (52.6%)28114944/45929032 bytes (61.2%)31973376/45929032 bytes (69.6%)35938304/45929032 bytes (78.2%)40026112/45929032 bytes (87.1%)43786240/45929032 bytes (95.3%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

In [16]:
# run many using range and zip them up for easier downloading
for i in range(5, 21):
  !python applications/vis_correspondence.py \
    --ckpt {ckpt_name} \
    --real_data_path {dataset_out} \
    --vis_in_stages \
    --real_size 512 \
    --output_resolution 512 \
    --resolution 512 \
    --label_path {mask_path} \
    --dset_indices {i}
  outname = f'gangealing-vis-out-{i}.zip'
  !zip -r {outname} visuals

100% 240/240 [00:05<00:00, 41.32it/s]
100% 240/240 [00:05<00:00, 41.54it/s]
100% 240/240 [00:05<00:00, 41.47it/s]
100% 240/240 [00:05<00:00, 41.58it/s]
Using /root/.cache/torch_extensions/py37_cu111 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py37_cu111/_splat/build.ninja...
Building extension module _splat...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module _splat...
tcmalloc: large alloc 1509949440 bytes == 0x5566d80de000 @  0x7ff6cf0c6b6b 0x7ff6cf0e6379 0x7ff5ce90bcde 0x7ff5ce90d452 0x7ff620934cf3 0x7ff620935867 0x7ff620c88cc1 0x7ff6209ca571 0x7ff6209cada5 0x7ff621096e52 0x7ff620f341db 0x7ff6209d3ebb 0x7ff62118bb02 0x7ff620c77dfe 0x7ff6ca0d9b2e 0x556595a954b0 0x556595a95240 0x556595b090f3 0x556595b039ee 0x556595a96bda 0x556595b04915 0x5565959d5d14 0x556595b05fe4 0x556595b03ced 0x556595a96bda 0x